In [1]:
import pandas as pd
import numpy as np

## Read Data

In [2]:
data = pd.read_csv('./Data/PURCHASE ORDER DATA EXTRACT 2012-2015_0.csv')

In [3]:
data.columns

Index(['Creation Date', 'Purchase Date', 'Fiscal Year', 'LPA Number',
       'Purchase Order Number', 'Requisition Number', 'Acquisition Type',
       'Sub-Acquisition Type', 'Acquisition Method', 'Sub-Acquisition Method',
       'Department Name', 'Supplier Code', 'Supplier Name',
       'Supplier Qualifications', 'Supplier Zip Code', 'CalCard', 'Item Name',
       'Item Description', 'Quantity', 'Unit Price', 'Total Price',
       'Classification Codes', 'Normalized UNSPSC', 'Commodity Title', 'Class',
       'Class Title', 'Family', 'Family Title', 'Segment', 'Segment Title',
       'Location'],
      dtype='object')

In [4]:
data['Creation Date']

0         08/27/2013
1         01/29/2014
2         11/01/2013
3         06/13/2014
4         03/12/2014
             ...    
346013    11/03/2014
346014    08/14/2014
346015    05/09/2014
346016    01/14/2014
346017    10/09/2013
Name: Creation Date, Length: 346018, dtype: object

In [5]:
data['Purchase Date']

0                NaN
1                NaN
2                NaN
3         06/05/2014
4         03/12/2014
             ...    
346013    09/25/2014
346014    07/20/2011
346015           NaN
346016    01/14/2014
346017    09/09/2013
Name: Purchase Date, Length: 346018, dtype: object

In [6]:
# !pip install --user transformers
# !pip install --user trl
# !pip install --user transformers[torch]
# !pip install --user accelerate -U

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

C:\Users\mnsah\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mnsah\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
from trl import SFTTrainer

In [9]:
from transformers import AutoModelForCausalLM
from datasets import load_dataset, Dataset
from trl import SFTTrainer

dataset = Dataset.from_pandas(pd.read_csv('./Data/hugging_face_train.csv'))

# model = AutoModelForCausalLM.from_pretrained("bert-base-cased", is_decoder=True)
model = AutoModelForCausalLM.from_pretrained("distilgpt2", is_decoder=True)

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/342557 [00:00<?, ? examples/s]

In [10]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.386500
1000,0.990000
1500,0.888300
2000,0.834400
2500,0.791400
3000,0.763200
3500,0.732100
4000,0.715700
4500,0.702700
5000,0.684000


Checkpoint destination directory tmp_trainer\checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=128460, training_loss=0.4905246533282947, metrics={'train_runtime': 15960.2981, 'train_samples_per_second': 64.389, 'train_steps_per_second': 8.049, 'total_flos': 8.558599787576525e+16, 'train_loss': 0.4905246533282947, 'epoch': 3.0})

In [11]:
# !nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3080 Ti (UUID: GPU-026b7bf3-29a1-ae9c-3fea-f82d5ba3d9b5)


In [79]:
peft_model_id="gpt2_finetuned"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('gpt2_finetuned\\tokenizer_config.json',
 'gpt2_finetuned\\special_tokens_map.json',
 'gpt2_finetuned\\vocab.json',
 'gpt2_finetuned\\merges.txt',
 'gpt2_finetuned\\added_tokens.json',
 'gpt2_finetuned\\tokenizer.json')

## Evaluate the test data

In [80]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [81]:
model = AutoModelForCausalLM.from_pretrained(f'./{peft_model_id}')
tokenizer = AutoTokenizer.from_pretrained(f'./{peft_model_id}')
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [82]:
dataset_test = Dataset.from_pandas(pd.read_csv('./Data/hugging_face_test.csv'))

In [83]:
# dataset_test['text']

In [84]:
from random import randrange

In [119]:
sample = dataset_test['text'][randrange(len(dataset_test["text"]))]
print(f'{sample}')

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
what are the values of Creation Date,Fiscal Year,Purchase Order Number,Requisition Number,Sub-Acquisition Type,Supplier Code,Supplier Name,Unit Price,Total Price,

### Input:
Purchase Date 12/24/2014 LPA Number 1-14-84-07 Acquisition Type NON-IT Goods Acquisition Method Statewide Contract Sub-Acquisition Method nan Department Name Corrections and Rehabilitation, Department of Supplier Qualifications nan Supplier Zip Code 91761 CalCard NO Item Name carrier replacement Item Description vest Quantity 8.0 Classification Codes 46151504 Normalized UNSPSC 46151504.0 Commodity Title Body armour Class 46151500.0 Class Title Crowd control equipment Family 46150000.0 Family Title Law enforcement Segment 46000000.0 Segment Title Defense and Law Enforcement and Security and Safety Equipment and Supplies Location 91761
(34.037

In [120]:
ind = sample.find('### Response:') + 13

In [121]:
input_ids = tokenizer(sample[:ind], return_tensors="pt", truncation=True)#.input_ids.cuda()

In [126]:
import torch
with torch.no_grad():
    outputs = tokenizer.batch_decode(model.generate(**input_ids, 
                                                     max_new_tokens=40, 
                                                     do_sample=True, 
                                                     top_p=0.9, 
                                                     temperature=0.9
                                                   ),
                                     skip_special_tokens=True,
                                    pad_token_id = '')[0]
    #return_dict_in_generate=True, output_scores=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [127]:
ind_out = outputs.find('### Response:')

In [128]:
print(f'{outputs[ind_out:]}')

### Response:
12/24/2014, 2014-2015, 4500261172, nan, nan, 1017088.0, Safariland, LLC, $35.00, $300


In [125]:
# outputs